In [1]:
import pandas as pd
import os

In [2]:
figsize_x = 14
figsize_y = 9

In [3]:
file_list = os.listdir("..\\all\\")
file_list.sort()
result_path = '..\\result\\'
csv_path = '..\\all\\'
filename = ''

In [4]:
def read_file(system_id, ess_id):
    global filename
    filename = 'ess_' + system_id + '_' + ess_id + '.csv'
    df = pd.read_csv(csv_path + filename, parse_dates=['C.target_date'], index_col = ['C.target_date'])
    return df

In [5]:
def accumulate_charge(df):
    global filename, result_path 
    df['C.CHG_A'] = df.index
    df['C.DIS_A'] = df.index
    df['M.CHG_A'] = df.index
    df['M.DIS_A'] = df.index
    df['M.CHG_P_A'] = df.index
    df['M.DIS_P_A'] = df.index
    
    for index, row in df.iterrows() :
        if (index == 0):
            if (row['C.batt1_status'] == 'CHG'):
                df.loc[index,'C.CHG_A'] = df.loc[index,'C.batt_p']
                df.loc[index,'C.DIS_A'] = 0
            if (row['C.batt1_status'] == 'DCHG'):
                df.loc[index,'C.DIS_A'] = df.loc[index,'C.batt_p']
                df.loc[index,'C.CHG_A'] = 0
            df.loc[index,'M.CHG_P_A'] = df.loc[index, 'M.batt1_chg_p']
            df.loc[index,'M.DIS_P_A'] = df.loc[index, 'M.batt1_dis_p']
            df.loc[index,'M.CHG_A'] = df.loc[index, 'M.batt1_chg']
            df.loc[index,'M.DIS_A'] = df.loc[index, 'M.batt1_dis']
        if (index > 0):
            if (row['C.batt1_status'] == 'CHG'):
                df.loc[index,'C.CHG_A'] = df.loc[index - 1,'C.CHG_A'] + df.loc[index,'C.batt_p']
                df.loc[index,'C.DIS_A'] = df.loc[index - 1,'C.DIS_A']
            if (row['C.batt1_status'] == 'DCHG'):
                df.loc[index,'C.DIS_A'] = df.loc[index - 1,'C.DIS_A'] + df.loc[index,'C.batt_p']
                df.loc[index,'C.CHG_A'] = df.loc[index - 1,'C.CHG_A']
            if (row['C.batt1_status'] == 'WAIT'):
                df.loc[index,'C.DIS_A'] = df.loc[index - 1,'C.DIS_A']
                df.loc[index,'C.CHG_A'] = df.loc[index - 1,'C.CHG_A']
            df.loc[index,'M.CHG_P_A'] = df.loc[index, 'M.batt1_chg_p'] + df.loc[index-1,'M.CHG_P_A']
            df.loc[index,'M.DIS_P_A'] = df.loc[index, 'M.batt1_dis_p'] + df.loc[index-1,'M.DIS_P_A']
            df.loc[index,'M.CHG_A'] = df.loc[index, 'M.batt1_chg'] + df.loc[index-1,'M.CHG_A']
            df.loc[index,'M.DIS_A'] = df.loc[index, 'M.batt1_dis'] + df.loc[index-1,'M.DIS_A']  
    
    df.to_csv(result_path + 'accumulate_' + filename)

In [6]:
def plot_soh_temp(df):
    import matplotlib.pyplot as plt
    
    fig = plt.figure(figsize=(figsize_x,figsize_y))

    ax1 = fig.add_subplot(111)

    ax1.plot(df.index, df['M.batt1_tmp'], color='y', label='Temperature')
    ax2 = ax1.twinx()
    ax2.plot(df.index, df['C.batt1_soh'], color='g', label='SOH')

    ax1.set_yticks([5, 10, 15, 20, 25, 30, 35, 40])
    ax2.set_yticks([80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])
    ax1.set_ylim(bottom=10, top=40)
    ax2.set_ylim(bottom=80, top=103)
    ax1.set_xlim(left=0, right=70000)

    ax1.legend(loc=1)
    ax2.legend(loc=2)

    ax1.set_title(filename + ' SOH & Battery Temperature')
    #plt.show()
    fig.savefig(result_path + filename + ' SOH & BAT Temperature.png')

In [7]:
def plot_soh_charge(df):
    import matplotlib.pyplot as plt

    fig = plt.figure(figsize=(figsize_x,figsize_y))

    ax1 = fig.add_subplot(111)

    ax1.plot(df.index, df['M.CHG_A'], color='r', label='CHG Accumulated')
    ax1.plot(df.index, df['M.DIS_A'], color='b', label='DISCHG Accumulated')
    ax2 = ax1.twinx()
    ax2.plot(df.index, df['C.batt1_soh'], color='g', label='SOH')

    ax1.set_yticks([1000000,2000000,3000000,4000000])
    ax2.set_yticks([80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])
    ax1.set_ylim(bottom=0, top=4000000)
    ax2.set_ylim(bottom=80, top=103)
    ax1.set_xlim(left=0, right=70000)

    ax1.legend(loc=1)
    ax2.legend(loc=2)

    ax1.set_title(filename + ' SOH & Charge Accumulated')
    #plt.show()
    fig.savefig(result_path + filename + ' SOH & Charge.png')

In [8]:
def plot_temp_charge(df):
    import matplotlib.pyplot as plt

    fig = plt.figure(figsize=(figsize_x,figsize_y))

    ax1 = fig.add_subplot(111)


    ax1.plot(df.index, df['M.batt1_tmp'], color='y', label='Temperature')
    ax2 = ax1.twinx()
    ax2.plot(df.index, df['M.CHG_A'], color='r', label='CHG Accumulated')
    ax2.plot(df.index, df['M.DIS_A'], color='b', label='DISCHG Accumulated')

    ax2.set_yticks([1000000,2000000,3000000,4000000])
    ax1.set_yticks([5, 10, 15, 20, 25, 30, 35, 40])
    ax2.set_ylim(bottom=0, top=4000000)
    ax1.set_ylim(bottom=10, top=40)
    ax1.set_xlim(left=0, right=70000)

    ax1.legend(loc=2)
    ax2.legend(loc=1)

    ax1.set_title(filename + ' Temperature & Charge')
    #plt.show()
    fig.savefig(result_path + filename + ' Temperature & Charge.png')

In [9]:
def plot_soc_charge(df):
    import matplotlib.pyplot as plt

    fig = plt.figure(figsize=(figsize_x,figsize_y))

    ax1 = fig.add_subplot(111)


    ax1.plot(df.index, df['M.batt1_soc'], color='y', label='SOC')
    ax2 = ax1.twinx()
    ax2.plot(df.index, df['M.CHG_A'], color='r', label='CHG Accumulated')
    ax2.plot(df.index, df['M.DIS_A'], color='b', label='DISCHG Accumulated')

    ax2.set_yticks([1000000,2000000,3000000,4000000])
    ax1.set_yticks([0, 25, 50, 75, 100])
    ax2.set_ylim(bottom=0, top=4000000)
    ax1.set_ylim(bottom=-50, top=150)
    ax1.set_xlim(left=0, right=70000)

    ax1.legend(loc=2)
    ax2.legend(loc=1)

    ax1.set_title(filename + ' SOC & Charge')
    #plt.show()
    fig.savefig(result_path + filename + ' SOC & Charge.png')

In [10]:
def plot_soc_soh(df):
    import matplotlib.pyplot as plt

    fig = plt.figure(figsize=(figsize_x,figsize_y))

    ax1 = fig.add_subplot(111)

    ax1.plot(df.index, df['M.batt1_soc'], color='y', label='SOC')
    ax2 = ax1.twinx()
    ax2.plot(df.index, df['C.batt1_soh'], color='g', label='SOH')

    ax1.set_yticks([0, 25, 50, 75, 100])
    ax2.set_yticks([80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])
    ax1.set_ylim(bottom=-50, top=150)
    ax2.set_ylim(bottom=80, top=103)
    ax1.set_xlim(left=0, right=70000)

    ax1.legend(loc=1)
    ax2.legend(loc=2)

    ax1.set_title(filename + ' SOC & SOH')
    #plt.show()
    fig.savefig(result_path + filename + ' SOC & SOH.png')

In [11]:
for i in range(len(file_list)):
    system_id = file_list[i].split('_')[1]
    ess_id = file_list[i].split('_')[2].split('.')[0]
    df = read_file(system_id, ess_id)
    accumulate_charge(df)
    plot_soh_charge(df)
    plot_soh_temp(df)
    plot_temp_charge(df)
    plot_soc_charge(df)
    plot_soc_soh(df)

C:\Users\sopark\AppData\Local\conda\conda\envs\idp3\lib\site-packages\matplotlib\pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


KeyboardInterrupt: 

In [ ]:
#df.head()

In [ ]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(figsize_x,figsize_y))

# ax1 = fig.add_subplot(111)

# ax1.plot(df.index, df['M.CHG_A'], color='r', label='CHG Accumulated')
# ax1.plot(df.index, df['M.DIS_A'], color='b', label='DISCHG Accumulated')
# ax2 = ax1.twinx()
# ax2.plot(df.index, df['C.batt1_soh'], color='g', label='SOH')

# ax1.set_yticks([1000000,2000000,3000000,4000000])
# ax2.set_yticks([80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])
# ax1.set_ylim(bottom=0, top=4000000)
# ax2.set_ylim(bottom=80, top=103)
# ax1.set_xlim(left=0, right=70000)

# ax1.legend(loc=1)
# ax2.legend(loc=2)

# ax1.set_title(filename + ' SOH & Charge Accumulated')
# plt.show()
# fig.savefig(result_path + filename + ' SOH & Charge.png')
# #df[['M.CHG_A','C.batt1_soh']].plot(secondary_y='C.batt1_soh')

# #plt.show()

In [ ]:

# import matplotlib.pyplot as plt

# fig = plt.figure(figsize=(figsize_x,figsize_y))

# ax1 = fig.add_subplot(111)

# ax1.scatter(df['M.CHG_A'], df['C.batt1_soh'], color='r', label='CHG Accumulated')

# #ax1.set_yticks([1000000,2000000,3000000,4000000])
# #ax2.set_yticks([80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100])
# ax1.set_ylim(bottom=80, top=100)
# #ax2.set_ylim(bottom=80, top=103)
# ax1.set_xlim(left=0, right=4000000)

# ax1.legend(loc=1)

# ax1.set_title(filename + ' SOH vs Charge Accumulated')
# plt.show()
# fig.savefig(result_path + filename + ' SOH vs Charge Accumulated.png')
